In [1]:
import pandas as pd
import re
from konlpy.tag import Okt

In [6]:
df_not = pd.read_csv("C:/Users/82105/Downloads/divorce_not_related_true.csv", encoding="utf-8-sig") # 이혼 아님
df = pd.read_csv("C:/Users/82105/Downloads/merged_score.csv", encoding="utf-8-sig")   #이혼 

df['is_divorce'] = 1
df_not['is_divorce'] = 0

# 두 데이터프레임 합치기
combined_df = pd.concat([df, df_not], ignore_index=True)

cols = combined_df.columns
print(len(cols))   # 컬럼 개수 확인
print(cols)  

combined_df.to_csv("C:/Users/82105/Downloads/combined2.csv", index=False, encoding='utf-8-sig')
combined_df = pd.read_csv("C:/Users/82105/Downloads/combined.csv", encoding="utf-8-sig")

# 전처리를 적용할 컬럼을 확인합니다. 'input'과 'output' 컬럼이 중요해 보입니다.
# 혹시 데이터가 없는 경우(NaN)를 대비해 빈 문자열로 채워줍니다.
combined_df['input'] = combined_df['input'].fillna('')
combined_df['output'] = combined_df['output'].fillna('')

# 모델 3 : 답변 검색 모델을 위한 '통합' 텍스트 컬럼 생성
combined_df['text_combined'] = combined_df['input'].astype(str) + " " + combined_df['output'].astype(str)

print("전처리 전 데이터 샘플:")
print(combined_df[['input', 'output']].head())

21
Index(['score', 'confidence', 'classification', 'tier1_count', 'tier2_count',
       'tier3_count', 'keyword_density', 'text_length', 'reason', 'doc_id',
       'casetype', 'casenames', 'input', 'output', 'title', 'fileName',
       'announce_date', 'decision_date', 'response_date', 'row_index',
       'is_divorce'],
      dtype='object')


In [91]:
combined_df = pd.read_csv("C:/Users/82105/Downloads/combined_before_jeon.csv", encoding="utf-8-sig")

In [40]:
combined_df2.to_csv("C:/Users/82105/Downloads/combined_before_jeon.csv", index=False, encoding='utf-8-sig')

In [93]:
print("=== 이혼 관련 문서만 토픽 모델링 시작 ===")
divorce_df = combined_df[combined_df['is_divorce'] == 1].copy()
non_divorce_df = combined_df[combined_df['is_divorce'] == 0].copy()

print(f"📊 데이터 분할:")
print(f"  • 이혼 관련 문서: {len(divorce_df)}개")
print(f"  • 비이혼 관련 문서: {len(non_divorce_df)}개")
print(f"  • 전체: {len(combined_df)}개")

# 이혼 문서만으로 토픽 모델링
divorce_df['topic_label'] = -1  # 빈 컬럼 생성

 # 비이혼 문서는 topic_label = -1 유지
non_divorce_df['topic_label'] = -1  # 미분류로 유지

=== 이혼 관련 문서만 토픽 모델링 시작 ===
📊 데이터 분할:
  • 이혼 관련 문서: 741개
  • 비이혼 관련 문서: 3706개
  • 전체: 4447개


In [94]:
divorce_df.to_csv("C:/Users/82105/Downloads/divorce_df.csv", index=False, encoding='utf-8-sig')

# 1단계 : KoNLPy를 사용한 정교한 전처리

In [201]:
MINIMAL_STOPWORDS = [
    '것', '수', '때', '등', '들', '더', '이', '그', '저', '나', 
    '우리', '같', '또', '만', '년', '월', '일', '하다', '있다', '되다',

     '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다',
    '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해',
    '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자','제호',

     '없', '있', '하', '되', '않',  '행위',
    '나', '우리', '너', '당신', '같', '또', '것', '때', '등', 
    '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과', '사람',

    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지''하나요', '위해', '이혼'
]

# # 법률 전문용어는 무조건 포함
# LEGAL_KEYWORDS = [ '위자료', '재산분할', '양육권', '친권', '면접교섭', 
#     '협의이혼', '청구', '배상', '손해', '책임',
#     '차용금', '반환', '취소', '원상회복', '사해행위', '채권자', '배우자', '이혼사유',
#      '이혼', '사유', 
#     '혼인', '금전거래', '청구권',  '액수', '정해지', '부적법',
#     '혼인파탄', '파탄', '분할', '양육비', '면접',
#     '교섭', '협의', '조정신청', '손해배상', '부부', '배우자', '당사자', '사람', '개인', '상대방'
# ]

In [202]:
def preprocess_text(text):  # 전처리 함수 
    """
    한국어 텍스트를 입력받아 전처리하는 함수:
    1. '제3자' 형태의 단어를 임시 토큰으로 보호/복원하여 숫자를 보존.
    2. 불필요한 한글 이외의 문자를 제거.
    3. 형태소 분석 및 어간 추출.
    4. 불용어 및 1글자 단어 제거.
    """
    # 1단계: 한글, 공백을 제외한 모든 문자 제거
    # re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)는 한글과 공백만 남기고 나머지는 지우라는 의미
    # 제숫자 + 한글 글자 + 선택적 조사까지 포함
    # 딕셔너리를 함수 내부에 선언하여 매 호출마다 초기화 (핵심 수정 사항)
    protected_matches = {}

    def protect_term(match):
        # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
        token = f"###TOKEN{len(protected_matches)}###"
        protected_matches[token] = match.group(0)
        return match.group(0) # 일단 원본 단어 그대로 둔 채로 형태소 분석 진행
    def strip_josa(text):
    # 한글 명사 뒤 “의”, “가”, “을” 등 제거
        return re.sub(r'([가-힣]+)(의|가|를|은|는|과|와)$', r'\1', text)
    

    # 1단계: '제3자' 형태의 단어를 임시 토큰으로 치환하여 보호
    re.sub(r'(제\d+[가-힣]+)', protect_term, text)

    # 2단계: 한글, 공백, 보호 토큰의 문자(#)만 남기고 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣# ]', '', text)

    # 2단계: Okt 형태소 분석기를 이용한 토큰화 및 어간 추출(Stemming)
    # okt.pos(text, stem=True)는 문장을 (단어, 품사) 형태로 나누고, '하다', '먹다'처럼 원형으로 만들어줍니다.
    # 예: "먹었었고" -> ('먹다', 'Verb')
    # Okt 형태소 분석기 객체 생성
    okt = Okt()
    word_tokens = okt.pos(text, stem=True)

    # 3단계: 불용어 제거
    # 형태소 분석 후 의미 있는 단어만 추출하는 필터링 과정
    # 의미를 가지는 명사, 동사, 형용사, 부사 중에서 1글자 이상인 단어만 추출합니다.
    # 품사 태그가 'Josa'(조사), 'Eomi'(어미), 'Punctuation'(구두점) 등인 단어들을 제거합니다.
    meaningful_words = []
    for word, pos in word_tokens:
        if word in LEGAL_KEYWORDS:
            meaningful_words.append(word)
        elif pos in ['Noun',' Verb'] and len(word) > 1:
            token = strip_josa(word)
            if token not in MINIMAL_STOPWORDS:
                meaningful_words.append(token)
     # 6. 보호된 단어 강제 포함 및 복원 (핵심)
    # 보호 목록에 있던 단어들을 강제로 최종 리스트에 추가합니다.
    # (이미 리스트에 분해되어 들어갔을 수 있지만, 완벽한 복원을 위해 강제 추가)
    for original_term in protected_matches.values():
        # '제3자' 자체를 하나의 단어로 명시적으로 추가
        meaningful_words.append(original_term)
        
    # 7. 중복 제거 및 최종 문자열 반환
    # Set을 이용해 중복 제거 후 리스트로 변환
    final_words = list(set(meaningful_words))
    # 최종적으로 공백으로 연결된 문자열을 반환합니다. 
    # 모델에 따라 리스트 형태(meaningful_words)를 그대로 사용할 수도 있습니다.
    # 텍스트에서 분석에 의미 있는 핵심 단어들만 남긴 리스트 생성
    return ' '.join(final_words)

In [203]:
# --- 4. 각 목적에 맞게 전처리 컬럼 생성 ---
# 질문 의도 파악 모델용: 'input' 컬럼만 전처리
# 'input' 컬럼에 전처리 함수를 적용하여 새로운 'input_processed' 컬럼 생성
divorce_df['input_processed'] = divorce_df['input'].apply(preprocess_text)

In [204]:
divorce_df['combined_processed'] = divorce_df['text_combined'].apply(preprocess_text)

In [206]:
# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력
print("\n=== 전처리 전/후 비교 ===")
for i in range(5):
    print(f"원본 [{i}]: {divorce_df['input'].iloc[i]}")
    print(f"결과 [{i}]: {divorce_df['input_processed'].iloc[i]}\n")


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(divorce_df[['input', 'input_processed']].head())


=== 전처리 전/후 비교 ===
원본 [0]: 혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?
결과 [0]: 거래 차용 발생 청구권 관련 인정 조건 금전 혼인 반환

원본 [1]: 제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?
결과 [1]: 부부 부정행위 책임 일방 법적 지게 제3자가

원본 [2]: 민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?
결과 [2]: 규정 제조 사유 이혼 제840조 민법 부정 제1호에서 의미 재판

원본 [3]: 배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?
결과 [3]: 배우자 액수 부정행위 위자료

원본 [4]: 사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어떤 경우인가요?
결과 [4]: 법해 소송 원상회복 부적 청구 채권자 취소 사해행위 다른

=== 각 목적에 맞게 생성된 전처리 컬럼들 ===
                                               input  \
0    혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?   
1          제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?   
2   민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?   
3         배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?   
4  사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...   

                       input_processed  
0       거래 차용 발생 청구권 관련 인정 조건 금전 혼인 반환  
1             부부 부정행위 책임 일방 법적 지게 제3자가  
2  규정 제조 사유 이혼 제840조 민법 부정 제1

# 2단계: 최적화된 TF-IDF 벡터화

In [207]:
# 텍스트 데이터를 머신러닝 알고리즘이 처리할 수 있는 수치 벡터로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# TF-IDF Vectorizer 설정
vectorizer = TfidfVectorizer(
    max_features=5000,      # 상위 빈도 1000개 단어만 사용
    min_df=5,
    max_df=0.5,           # 80% 이상 문서에 나타나는 단어 제거
    lowercase=False,         # 소문자 변환 x
    ngram_range=(1, 2),     # 1-gram, 2-gram 모두 사용
    sublinear_tf=True,            # TF 값에 로그 스케일 적용 (성능 향상)
    token_pattern=r'[가-힣]{2,}', # 한글 2글자 이상
)

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(divorce_df['input_processed'])  # X는 (문서 수 × 단어 수) 크기의 희소 행렬

print(f"TF-IDF 행렬 크기: {tfidf_matrix.shape}")
print(f"문서 수: {tfidf_matrix.shape[0]}")
print(f"특성(단어) 수: {tfidf_matrix.shape[1]}")

# 특성 이름(단어들) 확인
# vectorizer.get_feature_names_out()로 어떤 단어가 벡터에 들어갔는지 확인 가능
feature_names = vectorizer.get_feature_names_out()
print(f"상위 10개 특성: {feature_names[:10]}")
print(f"상위 20개 특성: {feature_names[:20]}")

TF-IDF 행렬 크기: (741, 287)
문서 수: 741
특성(단어) 수: 287
상위 10개 특성: ['간주' '간통' '개입' '개입 생활' '결정' '결정 손해배상' '결정 요소' '결정 정신' '계약' '고려']
상위 20개 특성: ['간주' '간통' '개입' '개입 생활' '결정' '결정 손해배상' '결정 요소' '결정 정신' '계약' '고려' '고려 법원'
 '고려 청구' '고통' '고통 손해배상' '고통 정신' '고통 청구' '공동' '관계' '관계 대한' '관계 부정행위']


# 3단계: 토픽 개수 최적화
# 최적의 토픽 개수를 찾기 위해 여러 지표를 사용

In [166]:
# 4-1. LDA Perplexity 계산 (낮을수록 좋음)

# 토픽 개수별 Perplexity 계산
perplexity_scores = []
topic_range = range(2, 11)  # 2~10개 토픽 테스트

for n_topics in topic_range:
    lda = LatentDirichletAllocation(
        n_components=n_topics, 
        random_state=42,
        max_iter=100,
        doc_topic_prior=0.1,      # 문서-토픽 분포 조정
        topic_word_prior=0.01     # 토픽-단어 분포 조정
    )
    lda.fit(tfidf_matrix)
    perplexity = lda.perplexity(tfidf_matrix)
    perplexity_scores.append(perplexity)
    print(f"토픽 수 {n_topics}: Perplexity = {perplexity:.4f}")

# 최적의 토픽 수 선택
optimal_topics = topic_range[np.argmin(perplexity_scores)]
print(f"최적의 토픽 수 (Perplexity 기준): {optimal_topics}")

토픽 수 2: Perplexity = 35462.0094
토픽 수 3: Perplexity = 43615.2530
토픽 수 4: Perplexity = 60242.6583
토픽 수 5: Perplexity = 57516.5650
토픽 수 6: Perplexity = 63302.6877
토픽 수 7: Perplexity = 78152.4240
토픽 수 8: Perplexity = 80238.3525
토픽 수 9: Perplexity = 95862.0042
토픽 수 10: Perplexity = 84849.3090
최적의 토픽 수 (Perplexity 기준): 2


# 더 정확한 토픽 모델링을 위해 Gensim 라이브러리를 사용
# 고급 기법: Gensim LDA + Coherence Score

In [189]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# 텍스트를 토큰 리스트로 변환
texts = [text.split() for text in divorce_df['input_processed']]

# Dictionary 생성
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.8)

# Corpus 생성  
corpus = [dictionary.doc2bow(text) for text in texts]

# 최적 토픽 수 찾기 (Coherence Score 사용)
coherence_scores = []
models_dict = {}
topic_range = range(2, 12)

for num_topics in topic_range:
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary, 
        num_topics=num_topics,
        random_state=42,
        passes=20,
        alpha='auto',        # 자동 조정
        per_word_topics=True
        
    )
    
    coherence_model = CoherenceModel(
        model=lda_model, 
        texts=texts, 
        dictionary=dictionary, 
        coherence='c_v'
    )
    
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    models_dict[num_topics] = lda_model
    print(f"토픽 수 {num_topics}: Coherence = {coherence_score:.4f}")

# 상위 3개 후보 선정
top_3_indices = np.argsort(coherence_scores)[-3:]
top_3_topics = [topic_range[i] for i in top_3_indices]
print(f"\nCoherence 상위 3개: {top_3_topics}")


# 각 후보의 실제 토픽 내용 확인
for num_topics in top_3_topics:
    print(f"\n=== {num_topics}개 토픽 결과 ===")
    model = models_dict[num_topics]
    
    for topic_id in range(num_topics):
        terms = model.show_topic(topic_id, topn=8)
        words = [term[0] for term in terms]
        print(f"토픽 {topic_id}: {', '.join(words)}")

토픽 수 2: Coherence = 0.3097
토픽 수 3: Coherence = 0.3209
토픽 수 4: Coherence = 0.3330
토픽 수 5: Coherence = 0.3446
토픽 수 6: Coherence = 0.3491
토픽 수 7: Coherence = 0.3603
토픽 수 8: Coherence = 0.3724
토픽 수 9: Coherence = 0.3339
토픽 수 10: Coherence = 0.3382
토픽 수 11: Coherence = 0.3442

Coherence 상위 3개: [6, 7, 8]

=== 6개 토픽 결과 ===
토픽 0: 손해, 정신, 배상, 입증, 부정행위, 적용, 청구, 대한
토픽 1: 부부, 생활, 제3자가, 책임, 조건, 혼인, 침해, 법적
토픽 2: 청구, 위자료, 부정행위, 손해배상, 대한, 기준, 범위, 인정
토픽 3: 재산, 분할, 이혼, 청구, 고려, 요소, 양육비, 판단
토픽 4: 부정행위, 손해배상, 배우자, 책임, 제3자가, 조건, 부부, 일방
토픽 5: 관계, 혼인, 인정, 부정, 사실혼, 배우자, 파탄, 조건

=== 7개 토픽 결과 ===
토픽 0: 정신, 손해배상, 부정행위, 고통, 청구, 손해, 배상, 배우자
토픽 1: 재산, 분할, 이혼, 조건, 부부, 사해행위, 효력, 추정
토픽 2: 청구, 위자료, 부정행위, 손해배상, 범위, 기준, 결정, 대한
토픽 3: 요소, 고려, 청구, 양육비, 판단, 법원, 대한, 기준
토픽 4: 부정행위, 배우자, 손해배상, 조건, 책임, 청구, 법적, 부부
토픽 5: 관계, 혼인, 부정, 배우자, 사실혼, 인정, 파탄, 불법행위
토픽 6: 부부, 제3자가, 부정행위, 책임, 생활, 불법행위, 조건, 일방

=== 8개 토픽 결과 ===
토픽 0: 손해, 배상, 정신, 발생, 대한, 고통, 관계, 해소
토픽 1: 재산, 분할, 이혼, 조건, 효력, 사해행위, 부부, 인정
토픽 2: 청구, 손해배상, 부정행위, 위자료, 범위, 결정, 기준, 소송


# 4단계: LDA 토픽 모델링 실행

In [208]:
# 최적의 토픽 수로 LDA 모델 생성 
final_lda = LatentDirichletAllocation(
    n_components=5,    # 최적 토픽 수 사용
    random_state=42,
    max_iter=100,
    learning_method='batch',         # 또는 'online'
    doc_topic_prior=0.1,        # 문서-토픽 분포 조정
    topic_word_prior=0.01     # 토픽-단어 분포 조정
)

# LDA 모델 훈련 및 토픽 확률 계산
lda_result = final_lda.fit_transform(tfidf_matrix)

# 각 문서의 주요 토픽 결정 (확률이 가장 높은 토픽)
topic_assignments = np.argmax(lda_result, axis=1)

print(f"LDA 결과 형태: {lda_result.shape}")

LDA 결과 형태: (741, 5)


# 5단계: 토픽별 주요 단어 분석

In [209]:
# 토픽별 대표 단어 추출
print("=== 토픽별 주요 단어 ===")
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(final_lda.components_):
    # 각 토픽에서 가중치가 높은 상위 10개 단어 추출
    top_words_idx = topic.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    top_weights = [topic[i] for i in top_words_idx]
    
    print(f"\n토픽 {topic_idx}:")
    for word, weight in zip(top_words, top_weights):
        print(f"  {word}: {weight:.4f}")


=== 토픽별 주요 단어 ===

토픽 0:
  이혼: 17.2834
  재산: 11.8825
  의무: 11.4772
  분할: 10.9563
  조건: 9.3630
  요소: 8.4846
  양육비: 7.9223
  청구: 7.8003
  지급: 7.5813
  소송: 6.8999

토픽 1:
  부정행위: 25.4262
  조건: 25.3586
  손해배상: 23.2311
  부정행위 조건: 21.1535
  책임: 17.7814
  배우자: 16.5995
  정신: 15.2337
  인정: 14.7464
  자가: 14.6135
  청구: 13.4194

토픽 2:
  재산: 11.5920
  범위: 10.3312
  분할: 9.8304
  효력: 9.7343
  관계: 9.1080
  사실혼: 8.8724
  기준: 7.7579
  판단: 7.4158
  이혼소송: 7.1094
  법률: 6.5353

토픽 3:
  부정행위: 21.9430
  손해배상: 19.1830
  위자료: 17.6841
  배우자: 17.6053
  청구: 17.0722
  기준: 16.3779
  배우자 부정행위: 13.2264
  정신: 12.8199
  부정행위 기준: 12.6538
  대한: 11.5298

토픽 4:
  생활: 16.3974
  부부: 15.4108
  자가: 11.6718
  침해: 10.9257
  책임: 9.7229
  혼인: 9.4150
  불법행위: 9.1919
  부부 불법행위: 8.7234
  하나요: 8.5515
  일방: 6.7546


In [193]:
# 최적의 토픽 수로 LDA 모델 생성 
final_lda = LatentDirichletAllocation(
    n_components=6,    # 최적 토픽 수 사용
    random_state=42,
    max_iter=100,
    learning_method='batch',         # 또는 'online'
    doc_topic_prior=0.1,        # 문서-토픽 분포 조정
    topic_word_prior=0.01     # 토픽-단어 분포 조정
)

# LDA 모델 훈련 및 토픽 확률 계산
lda_result = final_lda.fit_transform(tfidf_matrix)

# 각 문서의 주요 토픽 결정 (확률이 가장 높은 토픽)
topic_assignments = np.argmax(lda_result, axis=1)

print(f"LDA 결과 형태: {lda_result.shape}")

LDA 결과 형태: (741, 6)


In [194]:
# 토픽별 대표 단어 추출
print("=== 토픽별 주요 단어 ===")
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(final_lda.components_):
    # 각 토픽에서 가중치가 높은 상위 10개 단어 추출
    top_words_idx = topic.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    top_weights = [topic[i] for i in top_words_idx]
    
    print(f"\n토픽 {topic_idx}:")
    for word, weight in zip(top_words, top_weights):
        print(f"  {word}: {weight:.4f}")


=== 토픽별 주요 단어 ===

토픽 0:
  의무: 13.4350
  지급: 9.6573
  법적: 8.0409
  양육비: 6.7887
  약정: 5.5690
  근거: 5.4483
  대한: 5.4039
  대한 기준: 5.1556
  관련: 5.1415
  부부 의무: 4.5447

토픽 1:
  조건: 26.7358
  부정행위 조건: 21.1535
  부정행위: 20.0830
  손해배상: 16.6737
  책임: 14.6157
  배우자: 13.3136
  자가: 12.6087
  배우자 부정행위: 12.4383
  일방: 10.8040
  조건 책임: 10.5110

토픽 2:
  혼인: 12.4904
  침해: 10.9257
  관계: 9.6127
  생활: 8.6037
  효력: 7.3829
  법률: 6.4168
  생활 침해: 6.1245
  부부: 5.9249
  파탄: 5.4593
  혼인 손해배상: 5.2935

토픽 3:
  손해배상: 25.4766
  부정행위: 24.1695
  청구: 21.2353
  배우자: 19.3219
  정신: 18.2584
  청구 손해배상: 18.2321
  위자료: 17.4927
  기준: 17.1495
  고통: 15.1484
  손해배상 정신: 14.4894

토픽 4:
  부부: 13.7209
  자가: 10.3482
  법적: 8.5741
  책임: 8.4037
  발생: 8.3516
  대해: 8.2373
  생활: 7.8038
  부부 불법행위: 7.4963
  부정행위 일방: 7.2992
  일방: 7.0558

토픽 5:
  재산: 23.4646
  분할: 20.7766
  이혼: 19.1640
  사실혼: 8.4449
  적용: 7.3631
  조건: 7.2685
  소송: 6.8551
  청구: 6.5493
  사해행위: 6.4323
  고려: 6.1148


In [ ]:
# 최적의 토픽 수로 LDA 모델 생성 
final_lda = LatentDirichletAllocation(
    n_components=5,    # 최적 토픽 수 사용
    random_state=42,
    max_iter=100,
    learning_method='batch',         # 또는 'online'
    doc_topic_prior=0.1,        # 문서-토픽 분포 조정
    topic_word_prior=0.01     # 토픽-단어 분포 조정
)

# LDA 모델 훈련 및 토픽 확률 계산
lda_result = final_lda.fit_transform(tfidf_matrix)

# 각 문서의 주요 토픽 결정 (확률이 가장 높은 토픽)
topic_assignments = np.argmax(lda_result, axis=1)

print(f"LDA 결과 형태: {lda_result.shape}")

# 7단계: 최종 토픽 라벨 할당

In [53]:
# topic_label 컬럼에 최종 결과 저장
divorce_df['topic_label'] = topic_assignments

# 결과 확인
print("최종 토픽 라벨 분포:")
print(divorce_df['topic_label'].value_counts())

# 토픽별 문서 예시 확인
for topic_idx in range(2):
    print(f"\n=== 토픽 {topic_idx} 문서 예시 ===")
    topic_docs = divorce_df[divorce_df['topic_label'] == topic_idx]['input_processed']
    for i, doc in enumerate(topic_docs.head(3)):
        print(f"{i+1}. {doc[:100]}...")

최종 토픽 라벨 분포:
topic_label
1    419
0    322
Name: count, dtype: int64

=== 토픽 0 문서 예시 ===
1. 부부 어떻다 부정행위 일방 책임 법적 지게 제3자가...
2. 위자료 배우자 액수 정해지다 부정행위 어떻다 사람과...
3. 배우자 어떻다 부정행위 책임 사람과 제3자는 법적...

=== 토픽 1 문서 예시 ===
1. 거래 차용 발생 청구권 관련 인정 조건 금전 혼인 반환...
2. 규정 제조 사유 이혼 제840조 민법 부정 행위 제1호에서 제호 의미 재판...
3. 법해 소송 어떻다 원상회복 부적 청구 채권자 취소 사해행위 다른...


# 8단계: K-Means 클러스터링 (비교 분석)

In [ ]:
# K-Means로도 클러스터링 수행 (비교용)
kmeans_model = KMeans(
    n_clusters=2,
    random_state=42,
    max_iter=300,
    n_init=10  # 안정화
)

kmeans_labels = kmeans_model.fit_predict(tfidf_matrix)
divorce_df['topic_label_kmeans'] = kmeans_labels

# LDA와 K-Means 결과 비교
comparison_df = pd.DataFrame({
    'document_id': range(len(divorce_df)),
    'lda_topic': topic_assignments,
    'kmeans_cluster': kmeans_labels,
    'input_text': divorce_df['input_processed'].apply(lambda x: x[:50] + '...' if len(x) > 50 else x)
})

print("LDA vs K-Means 결과 비교:")
print(comparison_df.head(10))

# 9단계: 결과 해석 및 토픽 명명

In [55]:
print("\n=== 토픽별 상세 분석 ===")
for topic_idx, topic in enumerate(final_lda.components_):
        top_words_idx = topic.argsort()[-15:][::-1]
        top_words = [feature_names[i] for i in top_words_idx]
        top_weights = [topic[i] for i in top_words_idx]

        print(f"\n토픽 {topic_idx} (문서 {sum(topic_assignments == topic_idx)}개):")
        for word, weight in zip(top_words[:10], top_weights[:10]):
            print(f"  {word}: {weight:.4f}")


=== 토픽별 상세 분석 ===

토픽 0 (문서 322개):
  부정행위: 15.2116
  어떻다: 12.3022
  위자료: 11.6667
  청구: 11.5418
  배우자: 11.4851
  손해배상: 10.3333
  부부: 9.3836
  정신: 8.3524
  인하다: 8.3454
  조건: 8.1660

토픽 1 (문서 419개):
  부정행위: 21.0835
  손해배상: 20.8283
  인하다: 16.9923
  청구: 16.3750
  조건: 16.1127
  책임: 15.6096
  인정: 15.5422
  어떻다: 15.4694
  부부: 13.6787
  인하다 손해배상: 12.4126


In [58]:
#  전체 데이터프레임 재조합
final_df = pd.concat([divorce_df, non_divorce_df], ignore_index=True)

In [61]:
print(f"\n📈 최종 토픽 분포:")
topic_counts = final_df['topic_label'].value_counts().sort_index()
for topic_id, count in topic_counts.items():
    if topic_id == -1:
        print(f"  토픽 {topic_id} (미분류): {count}개")
    else:
        print(f"  토픽 {topic_id}: {count}개")


📈 최종 토픽 분포:
  토픽 -1 (미분류): 3706개
  토픽 0: 322개
  토픽 1: 419개


In [ ]:
# 토픽별 해석 (수동으로 의미를 부여)
topic_interpretation = {}

num_topics = len([t for t in topic_counts.index if t != -1])

if num_topics == 2:
        topic_interpretation = {
            0: "재산분할 관련",
            1: "양육권/친권 관련", 
            -1: "비이혼 관련"
        }
# elif num_topics == 4:
#         topic_interpretation = {
#             0: "재산분할 관련",
#             1: "양육권/친권 관련",
#             2: "위자료 관련",
#             3: "이혼절차 관련",
#             -1: "비이혼 관련"
#         }
# elif num_topics == 5:
#         topic_interpretation = {
#             0: "재산분할 관련",
#             1: "양육권/친권 관련", 
#             2: "위자료 관련",
#             3: "이혼절차 관련",
#             4: "기타 이혼 법률",
#             -1: "비이혼 관련"
#         }
else:
        # 기본 해석
        for i in range(num_topics):
            topic_interpretation[i] = f"이혼 토픽 {i}"
        topic_interpretation[-1] = "비이혼 관련"

# 해석 추가
final_df['topic_interpretation'] = final_df['topic_label'].map(topic_interpretation)

print(f"\n🎯 토픽 해석:")
for topic_id, interpretation in topic_interpretation.items():
        count = topic_counts.get(topic_id, 0)
        print(f"  토픽 {topic_id}: {interpretation} ({count}개)")


# 10단계: 결과 저장 및 활용

In [ ]:
# 결과 확인
print("\n=== 최종 결과 확인 ===")
print(final_df[['is_divorce', 'topic_label', 'topic_interpretation']].value_counts())

# CSV 저장
final_df.to_csv('divorce_topic_modeling_results.csv', index=False, encoding='utf-8-sig')
print("\n💾 결과가 'divorce_topic_modeling_results.csv'로 저장되었습니다!")

In [ ]:
# 결과를 CSV 파일로 저장
final_results = combined_df[['input_processed', 'topic_label', 'topic_interpretation']].copy()
final_results.to_csv('topic_modeling_results.csv', index=False, encoding='utf-8-sig')

print("토픽 모델링 완료! 결과가 저장되었습니다.")
print(f"- 최적 토픽 수: {optimal_topics}")
print(f"- 토픽 라벨 분포:")
for topic_id, count in combined_df['topic_label'].value_counts().sort_index().items():
    interpretation = topic_interpretation.get(topic_id, "해석 필요")
    print(f"  토픽 {topic_id} ({interpretation}): {count}개 문서")


In [ ]:
from sklearn.model_selection import train_test_split

X = data['clean_text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 로지스틱 회귀 학습
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# 예측
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))
